In [ ]:
import pandas as pd
import pickle
import numpy as np
import re
from cognite.client import CogniteClient
from cognite.client.exceptions import CogniteAPIError
import types
import sys
import warnings
warnings.filterwarnings("ignore")

def is_str(x):
    return isinstance(x, (str, np.str_))   # covers Python and NumPy strings

def is_series(x):
    return isinstance(x, pd.Series)

In [ ]:
# make the client object
# this can be done in many different ways, pick the one you feel more comfortable with...

client = CogniteClient()

#### Retrieve Production Data

In [3]:
# ---- Compatibility shim (no __new__ override!) ----
class DummyIndex(pd.Index):
    pass

numeric_module = types.ModuleType("pandas.core.indexes.numeric")
numeric_module.Int64Index  = DummyIndex
numeric_module.UInt64Index = DummyIndex
numeric_module.Float64Index = DummyIndex
numeric_module.RangeIndex  = pd.RangeIndex  # this still exists

sys.modules['pandas.core.indexes.numeric'] = numeric_module

# ---- Load pickle ----
with open("GOM_production.pickle", "rb") as f:
    production_data = pickle.load(f)

In [ ]:
df_meters = client.raw.rows.retrieve_dataframe("bsee", "meters_headers", limit=-1)
df_meters = df_meters.reset_index().rename(columns={"index": "external_id"})

df_well = client.raw.rows.retrieve_dataframe("bsee", "well_headers", limit=-1)
df_well = df_well.reset_index().rename(columns={"index": "external_id"})

# normalize dtypes
df_well   = df_well.copy()
df_meters = df_meters.copy()
col = 'PLATFORM_ID'  # or 'platfid'
mask = df_well[col].notna()
df_well.loc[mask, col] = (
    'PLTF-' + df_well.loc[mask, col].astype(str).str.replace(r'^PLTF-', '', regex=True)
)
df_well['API_WELL_NUMBER'] = df_well['API_WELL_NUMBER'].astype(str).str.strip()
df_meters['PLATFORM_ID'] = df_meters['PLATFORM_ID'].astype(str).str.strip()
df_meters['METER_ID'] = (
    'EQ_MET-' 
    + df_meters['FMP_NUMBER'].astype(str).str.strip()
    + '-'
    + df_meters['METER_SERIAL_NUM'].astype(str).str.strip()
)

# all wells per platform (NO per-platform drop!)
well_on_platform = (
    df_well[['PLATFORM_ID','API_WELL_NUMBER','WELL_NAME']]
    .dropna(subset=['PLATFORM_ID','API_WELL_NUMBER'])
    .drop_duplicates()       # keeps ALL wells on each platform
)

# meters per platform (dedup exact repeats)
meters_on_platform = (
    df_meters[['PLATFORM_ID','METER_ID','FMP_MEAS_TYP_DESC']]
    .dropna(subset=['PLATFORM_ID','METER_ID'])
    .drop_duplicates()
)

# attach every (platform, meter) to EVERY well on that platform
df_meters_w_parents = (
    meters_on_platform
    .merge(well_on_platform, on='PLATFORM_ID', how='inner')
)

# optional: clean up / sanity
df_meters_w_parents.drop(columns=['external_id'], errors='ignore', inplace=True)

print("meters_on_platform:", len(meters_on_platform))
print("well_on_platform:", len(well_on_platform))
print("rows after attach (should be >= meters_on_platform):", len(df_meters_w_parents))

df_meters_w_parents.head()


meters_on_platform: 977
well_on_platform: 2831
rows after attach (should be >= meters_on_platform): 24491


In [10]:
all_prod = {k[:-2]: v for k, v in production_data.items()}

In [11]:
for k, rec in all_prod.items():
    df = rec['Production Data'].copy().sort_values('Production Date').reset_index(drop=True)

    # ensure numeric & safe denom
    cols = ['Oil (bbls)', 'Gas (bbls)', 'Water (bbls)', 'Days of Production']
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    days = df['Days of Production'].replace(0, np.nan)

    # per-day rates
    df['OIL_BBL_PER_DAY'] = (df['Oil (bbls)']   / days).replace([np.inf, -np.inf], np.nan).fillna(0)
    df['GAS_MCF_PER_DAY'] = (df['Gas (bbls)']   / days).replace([np.inf, -np.inf], np.nan).fillna(0)  # adjust col name if gas is stored as Mcf column
    df['WATER_BBL_PER_DAY'] = (df['Water (bbls)'] / days).replace([np.inf, -np.inf], np.nan).fillna(0)
    df['CUMULATIVE_BOE_PER_DAY'] = ((df['Oil (bbls)'] + df['Gas (bbls)']/5.8) / days)\
                                .replace([np.inf, -np.inf], np.nan).fillna(0)

    # keep full DF
    rec['Production Data'] = df

    # expose TS as (date, value) lists
    rec['OIL_BBL_PER_DAY'] = list(df[['Production Date', 'OIL_BBL_PER_DAY']].itertuples(index=False, name=None))
    rec['GAS_MCF_PER_DAY'] = list(df[['Production Date', 'GAS_MCF_PER_DAY']].itertuples(index=False, name=None))
    rec['WATER_BBL_PER_DAY'] = list(df[['Production Date', 'WATER_BBL_PER_DAY']].itertuples(index=False, name=None))
    rec['CUMULATIVE_BOE_PER_DAY_TS'] = list(df[['Production Date', 'CUMULATIVE_BOE_PER_DAY']].itertuples(index=False, name=None))

In [18]:
# -----------------------------
# 0) Normalize & classify meters
# -----------------------------
m = df_meters_w_parents.copy()
m['API_WELL_NUMBER'] = m['API_WELL_NUMBER'].astype(str)
m['desc_norm'] = m['FMP_MEAS_TYP_DESC'].astype(str).str.lower()

# classify to the three buckets we care about
m['class'] = np.select(
    [
        m['desc_norm'].str.contains(r'\bgas\s*royalty\b', regex=True),
        m['desc_norm'].str.contains(r'liquid\s*allocation|alloc\b', regex=True),
        m['desc_norm'].str.contains(r'liquid\s*royalty\b', regex=True),
    ],
    ['gas', 'liquid_alloc', 'liquid_royalty'],
    default='other'
)

# keep only relevant meter classes
m = m[m['class'].isin(['gas','liquid_alloc','liquid_royalty'])].copy()

# -----------------------------------
# 1) Build per-API meter assignments
# -----------------------------------
def pick_meters(g: pd.DataFrame):
    # unique, deterministic ordering
    lr = sorted(set(g.loc[g['class']=='liquid_royalty','METER_ID'].astype(str)))
    la = sorted(set(g.loc[g['class']=='liquid_alloc','METER_ID'].astype(str)))
    gas = sorted(set(g.loc[g['class']=='gas','METER_ID'].astype(str)))

    issues = []

    # GAS: exactly one gas royalty meter
    gas_meter = gas[0] if gas else None
    if not gas: issues.append('missing gas meter')
    if len(gas) > 1: issues.append('multiple gas meters; picking first')

    # OIL: MUST be a liquid royalty meter
    oil_meter = lr[0] if lr else None
    if not lr: issues.append('missing liquid royalty for oil')

    # WATER: prefer liquid allocation; else use SECOND liquid royalty
    water_meter = la[0] if la else (lr[1] if len(lr) >= 2 else None)
    if la and len(la) > 1:
        issues.append('multiple liquid allocation meters; picking first')
    if not la and len(lr) < 2:
        issues.append('need two liquid meters (2×LR or 1×LR + 1×LA)')

    return {'oil': oil_meter, 'gas': gas_meter, 'water': water_meter}, issues

assignments = {}
violations = {}

for api, g in m.groupby('API_WELL_NUMBER'):
    chosen, issues = pick_meters(g)
    assignments[api] = chosen
    if issues:
        violations[api] = issues

# -----------------------------
# 2) Well/platform metadata
# -----------------------------
# Prefer df_well for scalar metadata (dedupe to ensure one row per API)

df_well['API_NUMBER'] = df_well['API_WELL_NUMBER']
wmeta = (df_well.assign(API_WELL_NUMBER=df_well['API_WELL_NUMBER'].astype(str))
               .sort_values(['API_WELL_NUMBER'])
               .drop_duplicates('API_WELL_NUMBER', keep='first')
               .set_index('API_WELL_NUMBER')[['PLATFORM_ID','WELL_NAME', 'API_NUMBER']])



def meta(api, col):
    if api not in wmeta.index: return None
    val = wmeta.loc[api, col]
    if isinstance(val, pd.Series):
        val = val.iloc[0] if not val.empty else None
    return None if pd.isna(val) else str(val)

def slug(s, maxlen=120):
    if not s: return ""
    s = re.sub(r"[^A-Za-z0-9_.:-]+", "-", str(s))
    s = re.sub(r"-{2,}", "-", s).strip("-")
    return s[:maxlen]

# -----------------------------
# 3) TS naming & units (per-day)
# -----------------------------
UNIT_MAP = {
    'OIL_BBL_PER_DAY': 'bbl/day',
    'GAS_MCF_PER_DAY': 'Mcf/day',
    'WATER_BBL_PER_DAY': 'bbl/day',
    'CUMULATIVE_BOE_PER_DAY_TS': 'boe/day',       # no meter assigned to cumulative
}

UNIT_MAP2 = {
    'OIL_BBL_PER_DAY': 'volume_flow_rate:bbl_us-per-day',
    'GAS_MCF_PER_DAY': 'volume_flow_rate:mega-ft3-per-day',
    'WATER_BBL_PER_DAY': 'volume_flow_rate:bbl_us-per-day',
    'CUMULATIVE_BOE_PER_DAY_TS': 'volume_flow_rate:bbl_us-per-day',       # no meter assigned to cumulative
}

NAME_MAP = {
    'OIL_BBL_PER_DAY': 'Daily Oil Production',
    'GAS_MCF_PER_DAY': 'Daily Gas Production',
    'WATER_BBL_PER_DAY': 'Daily Water Production',
    'CUMULATIVE_BOE_PER_DAY_TS': 'Cumulative Daily Production',
}

# Map TS -> which fluid’s meter to use
TS_TO_FLUID = {
    'OIL_BBL_PER_DAY': 'oil',
    'GAS_MCF_PER_DAY': 'gas',
    'WATER_BBL_PER_DAY': 'water',
    'CUMULATIVE_BOE_PER_DAY_TS': None,   # explicitly no meter
}

# Map TS -> which fluid’s meter to use
TS_LOOKUP = {
    'OIL_BBL_PER_DAY': 'OIL_BBL_PER_DAY',
    'GAS_MCF_PER_DAY': 'GAS_MCF_PER_DAY',
    'WATER_BBL_PER_DAY': 'WATER_BBL_PER_DAY',
    'CUMULATIVE_BOE_PER_DAY_TS': 'CUMULATIVE_BOE_PER_DAY_TS'
}

# -----------------------------
# 4) Build the header table
# -----------------------------
header_rows = []

for api, rec in all_prod.items():
    platf = meta(api, 'PLATFORM_ID')
    well  = meta(api, 'WELL_NAME')
    wapi = meta(api, 'API_NUMBER')

    chosen = assignments.get(api, {'oil': None, 'gas': None, 'water': None})

    for ts_name, ts_val in rec.items():
        if ts_name == 'Production Data':
            wellboreId = str(rec['Production Data']['API Number'].iloc[0])
            continue

        # choose the meter for this TS (or None)
        fluid = TS_TO_FLUID.get(ts_name, None)
        meter_id = chosen.get(fluid) if fluid else None

        # uniqueId: ts + api [+ meter if present], never attach to cumulative
        parts = [ts_name, api]
        if meter_id is not None and not ts_name.startswith('CUMULATIVE'):
            parts.append(meter_id)
        unique_id = "_".join(slug(p) for p in parts)

        desc = f"{NAME_MAP.get(ts_name, ts_name)} time series for well {api}" + (f" ({well})" if well else "")

        tags = [ts_name.lower()]
        if well:  tags.append(well)
        if platf: tags.append(platf)

        header_rows.append({
            "uniqueId": unique_id,
            "name": NAME_MAP.get(ts_name, ts_name),
            "description": desc,
            "tags": tags,
            "aliases": [f"{ts_name}:{api}"] + ([f"{ts_name}:{api}:{meter_id}"] if meter_id else []),
            "sourceId": api,
            "sourceContext": "BSEE-Production_Data",
            "source": None,
            "sourceCreatedTime": None,
            "sourceUpdatedTime": None,
            "sourceCreatedUser": None,
            "sourceUpdatedUser": None,
            "isStep": False,
            "type": "DOUBLE",
            "sourceUnit": UNIT_MAP.get(ts_name, ""),
            # extras
            "platformId": platf,
            "wellName": well,
            "wellAPI": wapi,
            "meterId": meter_id,
            "cogUoM": UNIT_MAP2.get(ts_name, ""),
            "wellboreId": wellboreId,
            "tsLookup": TS_LOOKUP.get(ts_name, ""),
        })

header_table = pd.DataFrame(header_rows).drop_duplicates(subset=['uniqueId'])

# -----------------------------
# 5) (Optional) review violations
# -----------------------------
violations_df = (
    pd.DataFrame([(api, "; ".join(msgs)) for api, msgs in violations.items()],
                 columns=['API_WELL_NUMBER','violations'])
    if violations else pd.DataFrame(columns=['API_WELL_NUMBER','violations'])
)

print(f"Header rows: {len(header_table):,}")
print(f"Wells failing meter rule: {len(violations_df):,}")
print(f'{len(header_table[header_table["meterId"].notna()])} headers with a meterId')
print(f'{len(list(set(header_table["uniqueId"].tolist())))} out of {len(header_table)} uniqueIds are indeed unique...')
# display(violations_df.head())
header_table.head()

Header rows: 11,960
Wells failing meter rule: 2,231
6094 headers with a meterId
11960 out of 11960 uniqueIds are indeed unique...


,uniqueId,name,description,tags,aliases,sourceId,sourceContext,source,sourceCreatedTime,sourceUpdatedTime,...,isStep,type,sourceUnit,platformId,wellName,wellAPI,meterId,cogUoM,wellboreId,tsLookup
0,OIL_BBL_PER_DAY_6080740312_EQ_MET-20177183650-...,Daily Oil Production,Daily Oil Production time series for well 6080...,"[oil_bbl_per_day, LL004, PLTF-GB0388A (COOPE F...","[OIL_BBL_PER_DAY:6080740312, OIL_BBL_PER_DAY:6...",6080740312,BSEE-Production_Data,None,None,None,...,False,DOUBLE,bbl/day,PLTF-GB0388A (COOPE FPF)-24079,LL004,6080740312,EQ_MET-20177183650-SL149844,volume_flow_rate:bbl_us-per-day,608074031200,OIL_BBL_PER_DAY
1,GAS_MCF_PER_DAY_6080740312_EQ_MET-3017718DS00-...,Daily Gas Production,Daily Gas Production time series for well 6080...,"[gas_mcf_per_day, LL004, PLTF-GB0388A (COOPE F...","[GAS_MCF_PER_DAY:6080740312, GAS_MCF_PER_DAY:6...",6080740312,BSEE-Production_Data,None,None,None,...,False,DOUBLE,Mcf/day,PLTF-GB0388A (COOPE FPF)-24079,LL004,6080740312,EQ_MET-3017718DS00-A15HC080,volume_flow_rate:mega-ft3-per-day,608074031200,GAS_MCF_PER_DAY
2,WATER_BBL_PER_DAY_6080740312_EQ_MET-2017718365...,Daily Water Production,Daily Water Production time series for well 60...,"[water_bbl_per_day, LL004, PLTF-GB0388A (COOPE...","[WATER_BBL_PER_DAY:6080740312, WATER_BBL_PER_D...",6080740312,BSEE-Production_Data,None,None,None,...,False,DOUBLE,bbl/day,PLTF-GB0388A (COOPE FPF)-24079,LL004,6080740312,EQ_MET-20177183650-SL149845,volume_flow_rate:bbl_us-per-day,608074031200,WATER_BBL_PER_DAY
3,CUMULATIVE_BOE_PER_DAY_TS_6080740312,Cumulative Daily Production,Cumulative Daily Production time series for we...,"[cumulative_boe_per_day_ts, LL004, PLTF-GB0388...",[CUMULATIVE_BOE_PER_DAY_TS:6080740312],6080740312,BSEE-Production_Data,None,None,None,...,False,DOUBLE,boe/day,PLTF-GB0388A (COOPE FPF)-24079,LL004,6080740312,None,volume_flow_rate:bbl_us-per-day,608074031200,CUMULATIVE_BOE_PER_DAY_TS
4,OIL_BBL_PER_DAY_6080440199,Daily Oil Production,Daily Oil Production time series for well 6080...,"[oil_bbl_per_day, 001, PLTF-EB0165A-10297]",[OIL_BBL_PER_DAY:6080440199],6080440199,BSEE-Production_Data,None,None,None,...,False,DOUBLE,bbl/day,PLTF-EB0165A-10297,001,6080440199,None,volume_flow_rate:bbl_us-per-day,608044019900,OIL_BBL_PER_DAY


#### Create TimeSeries Datapoints

In [22]:
from cognite.client.data_classes.data_modeling import NodeId

ts_space = 'dmu_rmdm_instances'

for i in range(len(header_table)):
    try:
        wapi = header_table['sourceId'].iloc[i]
        ts_extId = header_table['uniqueId'].iloc[i]
        ts = header_table['tsLookup'].iloc[i]
        dps = all_prod[wapi][ts]
        print(f"Iteration {i}: Upserting production data for well {wapi}")
        client.time_series.data.insert_multiple(
            [
                {"instance_id": NodeId(space=ts_space, external_id=ts_extId), "datapoints": dps}
            ]
        )
        print(f"Successfully upserted {len(dps)} datapoints in {ts_extId}")
    except CogniteAPIError as e:
        print(f"ℹ️  Somthing wrong for iteration '{i}' ({wapi} well).")
        raise

Iteration 0: Upserting production data for well 6080740312
Successfully upserted 112 datapoints in OIL_BBL_PER_DAY_6080740312_EQ_MET-20177183650-SL149844
Iteration 1: Upserting production data for well 6080740312
Successfully upserted 112 datapoints in GAS_MCF_PER_DAY_6080740312_EQ_MET-3017718DS00-A15HC080
Iteration 2: Upserting production data for well 6080740312
Successfully upserted 112 datapoints in WATER_BBL_PER_DAY_6080740312_EQ_MET-20177183650-SL149845
Iteration 3: Upserting production data for well 6080740312
Successfully upserted 112 datapoints in CUMULATIVE_BOE_PER_DAY_TS_6080740312
Iteration 4: Upserting production data for well 6080440199
Successfully upserted 1 datapoints in OIL_BBL_PER_DAY_6080440199
Iteration 5: Upserting production data for well 6080440199
Successfully upserted 1 datapoints in GAS_MCF_PER_DAY_6080440199
Iteration 6: Upserting production data for well 6080440199
Successfully upserted 1 datapoints in WATER_BBL_PER_DAY_6080440199
Iteration 7: Upserting pro